In [127]:

from numpy import loadtxt
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import numpy as np
from sklearn.linear_model import LogisticRegression, RidgeClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import LinearSVC, SVC
import eli5


In [128]:
def classifier(num=0, rfnest=1000, rfdep=10, SVMC=10):
    if num==0:
        return RandomForestClassifier(n_estimators=rfnest,
                                     criterion='gini',
                                     max_depth=rfdep,
                                     min_samples_split=2,
                                     min_samples_leaf=1,
                                     min_weight_fraction_leaf=0.0,
                                     max_features=None,
                                     max_leaf_nodes=None,
                                     bootstrap=True,
                                     oob_score=False,
                                     n_jobs=1,
                                     class_weight='balanced')
    elif num == 1:
        return LogisticRegression(C=10000, penalty='l2',fit_intercept=False)
    elif num == 2:
        return LinearSVC(C=SVMC,fit_intercept=False)
    elif num ==3:
        return RidgeClassifier(alpha=2, fit_intercept=False)
    elif num == 4:
        return SVC(C=10000, gamma=0.01, kernel='rbf',degree=3)
    
    
def diff_prediction(true, pred):
    for i, (t, p) in enumerate(zip(true, pred)):
        if t != p:
            print("Error instance %d, True=%d, Pred=%d" %(i,t,p))
        

In [129]:
from IPython.core.display import display, HTML
show_html = lambda html: display(HTML(html))
show_html_expl = lambda expl, **kwargs: show_html(eli5.format_as_html(expl, include_styles=False, **kwargs))
show_html(eli5.format_html_styles())

In [130]:
data = loadtxt('CVIrf.csv', delimiter=';', skiprows=1)
fnames = ['round','cadence','vel','sLength','sTime','sLengthCV','sTimeCV','rhfx','lhfx','rhfy','lhfy','rhfz','lhfz']
ncl = 6
X = data[:, 1:-ncl]
XNN = X.copy()
sc = MinMaxScaler()
X= sc.fit_transform(X)

n_splits = 10

#  3 strides 3 clusters

In [131]:

yv = data[:, -ncl:]
print(yv.shape)
nlb = 0
y = yv[:,nlb].reshape(yv.shape[0])


(153, 6)


## Random Forest

In [132]:

skf = StratifiedKFold(n_splits=n_splits)
skf.get_n_splits(X, y)

lacc = []
for train_index, test_index in skf.split(X, y):
    clf= classifier(num=0, rfnest=500, rfdep=8)
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    clf.fit(X_train, y_train)
    prediction = clf.predict(X_test)
    acc = accuracy_score(y_test, prediction)
    lacc.append(acc)

print('ACC mean=', np.mean(lacc))

ACC mean= 0.7970238095238096


In [133]:
clf= classifier(num=0, rfnest=500, rfdep=8)

clf.fit(X, y)

y_hat = clf.predict(X)
print (classification_report(y_hat, y))
print (confusion_matrix(y_hat, y))
print('ACC=', accuracy_score(y, y_hat))

             precision    recall  f1-score   support

        1.0       1.00      1.00      1.00        50
        2.0       1.00      1.00      1.00        37
        3.0       1.00      1.00      1.00        66

avg / total       1.00      1.00      1.00       153

[[50  0  0]
 [ 0 37  0]
 [ 0  0 66]]
ACC= 1.0


In [134]:
expl = eli5.explain_weights(clf, feature_names=fnames)

show_html_expl(expl)

Weight,Feature
0.4512 ± 0.0958,rhfx
0.1411 ± 0.1849,rhfz
0.0975 ± 0.1984,lhfz
0.0617 ± 0.1070,sTimeCV
0.0516 ± 0.0863,lhfx
0.0367 ± 0.0874,cadence
0.0351 ± 0.1057,sLength
0.0302 ± 0.0792,sTime
0.0294 ± 0.0635,rhfy
0.0215 ± 0.0668,sLengthCV


## SVM

In [135]:
skf = StratifiedKFold(n_splits=n_splits)
skf.get_n_splits(X, y)

lacc = []
for train_index, test_index in skf.split(X, y):
    clf= classifier(num=2, SVMC=0.1)
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    clf.fit(X_train, y_train)
    prediction = clf.predict(X_test)
    acc = accuracy_score(y_test, prediction)
    lacc.append(acc)

print('ACC mean=', np.mean(lacc))

ACC mean= 0.802797619047619


In [136]:
clf= classifier(num=2, SVMC=0.1)

clf.fit(X, y)

y_hat = clf.predict(X)
print (classification_report(y_hat, y))
print (confusion_matrix(y_hat, y))
print('ACC=', accuracy_score(y, y_hat))

             precision    recall  f1-score   support

        1.0       0.88      0.72      0.79        61
        2.0       0.43      0.73      0.54        22
        3.0       0.98      0.93      0.96        70

avg / total       0.86      0.82      0.83       153

[[44 17  0]
 [ 5 16  1]
 [ 1  4 65]]
ACC= 0.8169934640522876


In [137]:
diff_prediction(y, y_hat)

Error instance 8, True=2, Pred=1
Error instance 27, True=1, Pred=2
Error instance 30, True=2, Pred=3
Error instance 31, True=2, Pred=3
Error instance 40, True=1, Pred=2
Error instance 45, True=1, Pred=2
Error instance 51, True=3, Pred=2
Error instance 53, True=2, Pred=3
Error instance 54, True=2, Pred=1
Error instance 55, True=2, Pred=1
Error instance 56, True=2, Pred=1
Error instance 57, True=2, Pred=1
Error instance 70, True=1, Pred=2
Error instance 71, True=1, Pred=2
Error instance 72, True=1, Pred=3
Error instance 74, True=2, Pred=1
Error instance 75, True=2, Pred=1
Error instance 76, True=2, Pred=1
Error instance 77, True=2, Pred=1
Error instance 90, True=2, Pred=1
Error instance 91, True=2, Pred=1
Error instance 114, True=2, Pred=1
Error instance 115, True=2, Pred=1
Error instance 116, True=2, Pred=1
Error instance 117, True=2, Pred=1
Error instance 119, True=2, Pred=1
Error instance 123, True=2, Pred=3
Error instance 140, True=2, Pred=1


In [138]:
expl = eli5.explain_weights(clf, feature_names=fnames)

show_html_expl(expl)

## Decision Tree

In [139]:
clf= DecisionTreeClassifier()

clf.fit(XNN, y)

y_hat = clf.predict(XNN)
print (classification_report(y_hat, y))
print (confusion_matrix(y_hat, y))
print('ACC=', accuracy_score(y, y_hat))
expl = eli5.explain_weights(clf, feature_names=fnames)

show_html_expl(expl)

             precision    recall  f1-score   support

        1.0       1.00      1.00      1.00        50
        2.0       1.00      1.00      1.00        37
        3.0       1.00      1.00      1.00        66

avg / total       1.00      1.00      1.00       153

[[50  0  0]
 [ 0 37  0]
 [ 0  0 66]]
ACC= 1.0


Weight,Feature
0.5045,rhfx
0.1222,rhfz
0.1219,cadence
0.0912,sTimeCV
0.0729,lhfz
0.0398,lhfx
0.0269,sTime
0.0207,lhfy
0,rhfy
0,sLengthCV


# 5 strides 3 clusters

In [140]:
yv = data[:, -ncl:]
print(yv.shape)
nlb = 1
y = yv[:,nlb].reshape(yv.shape[0])


(153, 6)


## Random Forest

In [141]:
skf = StratifiedKFold(n_splits=n_splits)
skf.get_n_splits(X, y)

lacc = []
for train_index, test_index in skf.split(X, y):
    clf= classifier(num=0, rfnest=500, rfdep=10)
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    clf.fit(X_train, y_train)
    prediction = clf.predict(X_test)
    acc = accuracy_score(y_test, prediction)
    lacc.append(acc)

print('ACC mean=', np.mean(lacc))

ACC mean= 0.7518040293040292


In [142]:
clf= classifier(num=0, rfnest=500, rfdep=10)

clf.fit(X, y)

y_hat = clf.predict(X)
print (classification_report(y_hat, y))
print (confusion_matrix(y_hat, y))
print('ACC=', accuracy_score(y, y_hat))

             precision    recall  f1-score   support

        1.0       1.00      1.00      1.00        59
        2.0       1.00      1.00      1.00        56
        3.0       1.00      1.00      1.00        38

avg / total       1.00      1.00      1.00       153

[[59  0  0]
 [ 0 56  0]
 [ 0  0 38]]
ACC= 1.0


In [143]:
expl = eli5.explain_weights(clf, feature_names=fnames)

show_html_expl(expl)

Weight,Feature
0.3746 ± 0.1018,rhfx
0.1628 ± 0.2167,lhfz
0.1436 ± 0.2312,rhfz
0.0685 ± 0.1496,lhfy
0.0460 ± 0.0905,cadence
0.0396 ± 0.0921,vel
0.0340 ± 0.0784,lhfx
0.0330 ± 0.1010,sTimeCV
0.0272 ± 0.0746,sLength
0.0230 ± 0.0673,sLengthCV


## SVM

In [144]:
skf = StratifiedKFold(n_splits=n_splits)
skf.get_n_splits(X, y)

lacc = []
for train_index, test_index in skf.split(X, y):
    clf= classifier(num=2, SVMC=0.1)
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    clf.fit(X_train, y_train)
    prediction = clf.predict(X_test)
    acc = accuracy_score(y_test, prediction)
    lacc.append(acc)

print('ACC mean=', np.mean(lacc))

ACC mean= 0.7959111721611721


In [145]:
clf= classifier(num=2, SVMC=0.1)

clf.fit(X, y)

y_hat = clf.predict(X)
print (classification_report(y_hat, y))
print (confusion_matrix(y_hat, y))
print('ACC=', accuracy_score(y, y_hat))

             precision    recall  f1-score   support

        1.0       0.98      0.87      0.92        67
        2.0       0.89      0.77      0.83        65
        3.0       0.37      0.67      0.47        21

avg / total       0.86      0.80      0.82       153

[[58  0  9]
 [ 0 50 15]
 [ 1  6 14]]
ACC= 0.7973856209150327


In [146]:
diff_prediction(y, y_hat)

Error instance 22, True=2, Pred=3
Error instance 23, True=2, Pred=3
Error instance 25, True=3, Pred=2
Error instance 28, True=3, Pred=1
Error instance 29, True=3, Pred=1
Error instance 30, True=3, Pred=1
Error instance 31, True=3, Pred=1
Error instance 37, True=1, Pred=3
Error instance 44, True=2, Pred=3
Error instance 45, True=2, Pred=3
Error instance 55, True=3, Pred=2
Error instance 56, True=3, Pred=2
Error instance 70, True=2, Pred=3
Error instance 71, True=2, Pred=3
Error instance 74, True=3, Pred=2
Error instance 75, True=3, Pred=2
Error instance 76, True=3, Pred=2
Error instance 77, True=3, Pred=2
Error instance 90, True=3, Pred=2
Error instance 91, True=3, Pred=2
Error instance 92, True=3, Pred=2
Error instance 114, True=3, Pred=2
Error instance 115, True=3, Pred=2
Error instance 117, True=3, Pred=2
Error instance 119, True=3, Pred=2
Error instance 123, True=3, Pred=1
Error instance 124, True=3, Pred=2
Error instance 149, True=3, Pred=1
Error instance 150, True=3, Pred=1
Error 

In [147]:
expl = eli5.explain_weights(clf, feature_names=fnames)

show_html_expl(expl)

## Decision tree

In [148]:
clf= DecisionTreeClassifier()

clf.fit(XNN, y)

y_hat = clf.predict(XNN)
print (classification_report(y_hat, y))
print (confusion_matrix(y_hat, y))
print('ACC=', accuracy_score(y, y_hat))
expl = eli5.explain_weights(clf, feature_names=fnames)

show_html_expl(expl)

             precision    recall  f1-score   support

        1.0       1.00      1.00      1.00        59
        2.0       1.00      1.00      1.00        56
        3.0       1.00      1.00      1.00        38

avg / total       1.00      1.00      1.00       153

[[59  0  0]
 [ 0 56  0]
 [ 0  0 38]]
ACC= 1.0


Weight,Feature
0.4323,rhfx
0.2291,lhfz
0.1017,cadence
0.0613,lhfx
0.0489,rhfz
0.0433,lhfy
0.0399,round
0.0377,sLength
0.0057,rhfy
0,sTimeCV


# 5 strides 5 clusters

In [149]:
yv = data[:, -ncl:]
print(yv.shape)
nlb = 2
y = yv[:,nlb].reshape(yv.shape[0])


(153, 6)


## Random Forest

In [150]:
skf = StratifiedKFold(n_splits=n_splits)
skf.get_n_splits(X, y)

lacc = []
for train_index, test_index in skf.split(X, y):
    clf= classifier(num=0, rfnest=1000, rfdep=8)
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    clf.fit(X_train, y_train)
    prediction = clf.predict(X_test)
    acc = accuracy_score(y_test, prediction)
    lacc.append(acc)

print('ACC mean=', np.mean(lacc))

ACC mean= 0.7922209653092006


In [151]:
clf= classifier(num=0, rfnest=1000, rfdep=8)

clf.fit(X, y)

y_hat = clf.predict(X)
print (classification_report(y_hat, y))
print (confusion_matrix(y_hat, y))
print('ACC=', accuracy_score(y, y_hat))

             precision    recall  f1-score   support

        1.0       1.00      1.00      1.00        30
        2.0       1.00      1.00      1.00        58
        3.0       1.00      1.00      1.00        18
        4.0       1.00      1.00      1.00        25
        5.0       1.00      1.00      1.00        22

avg / total       1.00      1.00      1.00       153

[[30  0  0  0  0]
 [ 0 58  0  0  0]
 [ 0  0 18  0  0]
 [ 0  0  0 25  0]
 [ 0  0  0  0 22]]
ACC= 1.0


In [152]:
expl = eli5.explain_weights(clf, feature_names=fnames)

show_html_expl(expl)

Weight,Feature
0.3821 ± 0.1143,rhfx
0.1403 ± 0.2120,rhfz
0.1162 ± 0.2006,lhfz
0.0865 ± 0.1562,sTimeCV
0.0604 ± 0.1148,sTime
0.0411 ± 0.0741,rhfy
0.0334 ± 0.0654,lhfx
0.0299 ± 0.0782,cadence
0.0290 ± 0.0709,sLengthCV
0.0274 ± 0.0659,vel


## SVM

In [153]:
skf = StratifiedKFold(n_splits=n_splits)
skf.get_n_splits(X, y)

lacc = []
for train_index, test_index in skf.split(X, y):
    clf= classifier(num=2, SVMC=10)
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    clf.fit(X_train, y_train)
    prediction = clf.predict(X_test)
    acc = accuracy_score(y_test, prediction)
    lacc.append(acc)

print('ACC mean=', np.mean(lacc))

ACC mean= 0.6994909502262443


In [154]:
clf= classifier(num=2, SVMC=10)

clf.fit(X, y)

y_hat = clf.predict(X)
print (classification_report(y_hat, y))
print (confusion_matrix(y_hat, y))
print('ACC=', accuracy_score(y, y_hat))

             precision    recall  f1-score   support

        1.0       0.80      0.73      0.76        33
        2.0       0.97      0.92      0.94        61
        3.0       0.67      0.75      0.71        16
        4.0       0.52      0.65      0.58        20
        5.0       0.82      0.78      0.80        23

avg / total       0.82      0.80      0.81       153

[[24  0  0  8  1]
 [ 0 56  4  0  1]
 [ 2  2 12  0  0]
 [ 4  0  1 13  2]
 [ 0  0  1  4 18]]
ACC= 0.803921568627451


In [155]:
diff_prediction(y, y_hat)

Error instance 26, True=4, Pred=5
Error instance 29, True=3, Pred=2
Error instance 30, True=3, Pred=2
Error instance 32, True=3, Pred=2
Error instance 36, True=2, Pred=3
Error instance 37, True=2, Pred=3
Error instance 41, True=4, Pred=5
Error instance 42, True=4, Pred=5
Error instance 44, True=4, Pred=5
Error instance 70, True=1, Pred=3
Error instance 71, True=1, Pred=3
Error instance 72, True=1, Pred=4
Error instance 73, True=1, Pred=4
Error instance 74, True=3, Pred=5
Error instance 76, True=5, Pred=4
Error instance 77, True=3, Pred=4
Error instance 84, True=4, Pred=1
Error instance 85, True=4, Pred=1
Error instance 90, True=4, Pred=1
Error instance 91, True=4, Pred=1
Error instance 92, True=4, Pred=1
Error instance 93, True=4, Pred=1
Error instance 114, True=4, Pred=1
Error instance 115, True=5, Pred=4
Error instance 123, True=5, Pred=2
Error instance 137, True=1, Pred=4
Error instance 138, True=4, Pred=1
Error instance 139, True=1, Pred=4
Error instance 145, True=5, Pred=1
Error i

In [156]:
expl = eli5.explain_weights(clf, feature_names=fnames)

show_html_expl(expl)

## Decision tree

In [157]:
clf= DecisionTreeClassifier()

clf.fit(XNN, y)

y_hat = clf.predict(XNN)
print (classification_report(y_hat, y))
print (confusion_matrix(y_hat, y))
print('ACC=', accuracy_score(y, y_hat))
expl = eli5.explain_weights(clf, feature_names=fnames)

show_html_expl(expl)

             precision    recall  f1-score   support

        1.0       1.00      1.00      1.00        30
        2.0       1.00      1.00      1.00        58
        3.0       1.00      1.00      1.00        18
        4.0       1.00      1.00      1.00        25
        5.0       1.00      1.00      1.00        22

avg / total       1.00      1.00      1.00       153

[[30  0  0  0  0]
 [ 0 58  0  0  0]
 [ 0  0 18  0  0]
 [ 0  0  0 25  0]
 [ 0  0  0  0 22]]
ACC= 1.0


Weight,Feature
0.4510,rhfx
0.2594,rhfz
0.0726,rhfy
0.0709,sLength
0.0389,lhfz
0.0365,sLengthCV
0.0201,lhfy
0.0155,round
0.0137,lhfx
0.0130,vel


# 6 strides 3 clusters

In [158]:
yv = data[:, -ncl:]
print(yv.shape)
nlb = 4
y = yv[:,nlb].reshape(yv.shape[0])


(153, 6)


## Random Forest

In [159]:
skf = StratifiedKFold(n_splits=n_splits)
skf.get_n_splits(X, y)

lacc = []
for train_index, test_index in skf.split(X, y):
    clf= classifier(num=0, rfnest=1000, rfdep=8)
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    clf.fit(X_train, y_train)
    prediction = clf.predict(X_test)
    acc = accuracy_score(y_test, prediction)
    lacc.append(acc)

print('ACC mean=', np.mean(lacc))

ACC mean= 0.7944871794871795


In [160]:
clf= classifier(num=0, rfnest=1000, rfdep=8)

clf.fit(X, y)

y_hat = clf.predict(X)
print (classification_report(y_hat, y))
print (confusion_matrix(y_hat, y))
print('ACC=', accuracy_score(y, y_hat))

             precision    recall  f1-score   support

        1.0       1.00      1.00      1.00        49
        2.0       1.00      1.00      1.00        58
        3.0       1.00      1.00      1.00        46

avg / total       1.00      1.00      1.00       153

[[49  0  0]
 [ 0 58  0]
 [ 0  0 46]]
ACC= 1.0


In [161]:
expl = eli5.explain_weights(clf, feature_names=fnames)

show_html_expl(expl)

Weight,Feature
0.4066 ± 0.0994,rhfx
0.1823 ± 0.2310,rhfz
0.1459 ± 0.2442,lhfz
0.0694 ± 0.1008,sTime
0.0439 ± 0.0943,sTimeCV
0.0267 ± 0.0660,lhfx
0.0262 ± 0.0798,vel
0.0244 ± 0.0630,rhfy
0.0238 ± 0.0769,sLength
0.0218 ± 0.0621,lhfy


## SVM

In [162]:
skf = StratifiedKFold(n_splits=n_splits)
skf.get_n_splits(X, y)

lacc = []
for train_index, test_index in skf.split(X, y):
    clf= classifier(num=2, SVMC=0.1)
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    clf.fit(X_train, y_train)
    prediction = clf.predict(X_test)
    acc = accuracy_score(y_test, prediction)
    lacc.append(acc)

print('ACC mean=', np.mean(lacc))

ACC mean= 0.753191391941392


In [163]:
clf= classifier(num=2, SVMC=0.1)

clf.fit(X, y)

y_hat = clf.predict(X)
print (classification_report(y_hat, y))
print (confusion_matrix(y_hat, y))
print('ACC=', accuracy_score(y, y_hat))

             precision    recall  f1-score   support

        1.0       0.47      0.74      0.58        31
        2.0       0.98      0.85      0.91        67
        3.0       0.85      0.71      0.77        55

avg / total       0.83      0.78      0.79       153

[[23  1  7]
 [10 57  0]
 [16  0 39]]
ACC= 0.7777777777777778


In [164]:
diff_prediction(y, y_hat)

Error instance 22, True=3, Pred=1
Error instance 27, True=3, Pred=1
Error instance 28, True=1, Pred=2
Error instance 29, True=1, Pred=2
Error instance 30, True=1, Pred=2
Error instance 31, True=1, Pred=2
Error instance 37, True=2, Pred=1
Error instance 39, True=1, Pred=2
Error instance 54, True=1, Pred=3
Error instance 55, True=1, Pred=3
Error instance 56, True=1, Pred=3
Error instance 57, True=1, Pred=3
Error instance 70, True=3, Pred=1
Error instance 71, True=3, Pred=1
Error instance 72, True=3, Pred=1
Error instance 73, True=3, Pred=1
Error instance 74, True=1, Pred=3
Error instance 75, True=1, Pred=3
Error instance 76, True=1, Pred=3
Error instance 77, True=1, Pred=3
Error instance 90, True=1, Pred=3
Error instance 91, True=1, Pred=3
Error instance 92, True=1, Pred=3
Error instance 110, True=3, Pred=1
Error instance 114, True=1, Pred=3
Error instance 115, True=1, Pred=3
Error instance 116, True=1, Pred=3
Error instance 117, True=1, Pred=3
Error instance 118, True=1, Pred=3
Error in

In [165]:
expl = eli5.explain_weights(clf, feature_names=fnames)

show_html_expl(expl)

## Decision tree

In [166]:
clf= DecisionTreeClassifier()

clf.fit(XNN, y)

y_hat = clf.predict(XNN)
print (classification_report(y_hat, y))
print (confusion_matrix(y_hat, y))
print('ACC=', accuracy_score(y, y_hat))
expl = eli5.explain_weights(clf, feature_names=fnames)

show_html_expl(expl)

             precision    recall  f1-score   support

        1.0       1.00      1.00      1.00        49
        2.0       1.00      1.00      1.00        58
        3.0       1.00      1.00      1.00        46

avg / total       1.00      1.00      1.00       153

[[49  0  0]
 [ 0 58  0]
 [ 0  0 46]]
ACC= 1.0


Weight,Feature
0.4221,rhfx
0.1616,rhfz
0.1406,sTime
0.0938,lhfz
0.0658,lhfx
0.0424,sLength
0.0333,sTimeCV
0.0179,vel
0.0131,lhfy
0.0093,rhfy


# 6 strides 4 clusters

In [167]:
yv = data[:, -ncl:]
print(yv.shape)
nlb = 5
y = yv[:,nlb].reshape(yv.shape[0])


(153, 6)


## Random Forest

In [168]:
skf = StratifiedKFold(n_splits=n_splits)
skf.get_n_splits(X, y)

lacc = []
for train_index, test_index in skf.split(X, y):
    clf= classifier(num=0, rfnest=1000, rfdep=8)
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    clf.fit(X_train, y_train)
    prediction = clf.predict(X_test)
    acc = accuracy_score(y_test, prediction)
    lacc.append(acc)

print('ACC mean=', np.mean(lacc))

ACC mean= 0.8031372549019608


In [169]:
clf= classifier(num=0, rfnest=1000, rfdep=8)

clf.fit(X, y)

y_hat = clf.predict(X)
print (classification_report(y_hat, y))
print (confusion_matrix(y_hat, y))
print('ACC=', accuracy_score(y, y_hat))

             precision    recall  f1-score   support

        1.0       1.00      1.00      1.00        32
        2.0       1.00      1.00      1.00        41
        3.0       1.00      1.00      1.00        23
        4.0       1.00      1.00      1.00        57

avg / total       1.00      1.00      1.00       153

[[32  0  0  0]
 [ 0 41  0  0]
 [ 0  0 23  0]
 [ 0  0  0 57]]
ACC= 1.0


In [170]:
expl = eli5.explain_weights(clf, feature_names=fnames)

show_html_expl(expl)

Weight,Feature
0.3365 ± 0.1090,rhfx
0.2265 ± 0.2278,rhfz
0.1016 ± 0.2017,lhfz
0.0882 ± 0.1580,sTimeCV
0.0614 ± 0.1101,cadence
0.0336 ± 0.0726,lhfx
0.0327 ± 0.0817,sLengthCV
0.0324 ± 0.0724,rhfy
0.0276 ± 0.0667,sLength
0.0220 ± 0.0580,sTime


## SVM

In [171]:
skf = StratifiedKFold(n_splits=n_splits)
skf.get_n_splits(X, y)

lacc = []
for train_index, test_index in skf.split(X, y):
    clf= classifier(num=2, SVMC=50)
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    clf.fit(X_train, y_train)
    prediction = clf.predict(X_test)
    acc = accuracy_score(y_test, prediction)
    lacc.append(acc)

print('ACC mean=', np.mean(lacc))

ACC mean= 0.682561858076564


In [172]:
clf= classifier(num=2, SVMC=50)

clf.fit(X, y)

y_hat = clf.predict(X)
print (classification_report(y_hat, y))
print (confusion_matrix(y_hat, y))
print('ACC=', accuracy_score(y, y_hat))

             precision    recall  f1-score   support

        1.0       0.78      0.74      0.76        34
        2.0       0.83      0.81      0.82        42
        3.0       0.70      0.80      0.74        20
        4.0       0.95      0.95      0.95        57

avg / total       0.85      0.84      0.84       153

[[25  6  3  0]
 [ 7 34  1  0]
 [ 0  1 16  3]
 [ 0  0  3 54]]
ACC= 0.8431372549019608


In [173]:
diff_prediction(y, y_hat)

Error instance 27, True=1, Pred=2
Error instance 30, True=3, Pred=4
Error instance 32, True=3, Pred=4
Error instance 36, True=4, Pred=3
Error instance 37, True=4, Pred=3
Error instance 38, True=4, Pred=3
Error instance 39, True=3, Pred=4
Error instance 48, True=2, Pred=3
Error instance 71, True=1, Pred=2
Error instance 72, True=1, Pred=2
Error instance 73, True=1, Pred=2
Error instance 74, True=3, Pred=1
Error instance 75, True=3, Pred=2
Error instance 76, True=3, Pred=1
Error instance 77, True=3, Pred=1
Error instance 84, True=2, Pred=1
Error instance 90, True=2, Pred=1
Error instance 91, True=2, Pred=1
Error instance 92, True=2, Pred=1
Error instance 93, True=2, Pred=1
Error instance 114, True=2, Pred=1
Error instance 127, True=1, Pred=2
Error instance 130, True=1, Pred=2
Error instance 137, True=1, Pred=2


In [174]:
expl = eli5.explain_weights(clf, feature_names=fnames)

show_html_expl(expl)

## Decision Tree

In [175]:
clf= DecisionTreeClassifier()

clf.fit(XNN, y)

y_hat = clf.predict(XNN)
print (classification_report(y_hat, y))
print (confusion_matrix(y_hat, y))
print('ACC=', accuracy_score(y, y_hat))
expl = eli5.explain_weights(clf, feature_names=fnames)

show_html_expl(expl)

             precision    recall  f1-score   support

        1.0       1.00      1.00      1.00        32
        2.0       1.00      1.00      1.00        41
        3.0       1.00      1.00      1.00        23
        4.0       1.00      1.00      1.00        57

avg / total       1.00      1.00      1.00       153

[[32  0  0  0]
 [ 0 41  0  0]
 [ 0  0 23  0]
 [ 0  0  0 57]]
ACC= 1.0


Weight,Feature
0.4319,rhfx
0.1658,lhfz
0.1494,rhfz
0.0556,lhfx
0.0460,sLength
0.0396,sTimeCV
0.0331,sTime
0.0311,cadence
0.0176,sLengthCV
0.0163,rhfy
